# **ME 193, Spring 2021**
## Final Project Notebook 


**Notes**:

1. 

In [70]:
import os
import numpy as np
import pandas as pd
from IPython.display import JSON
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.tree as tree
import sklearn.metrics as mt
from sklearn.metrics import classification_report
import sklearn.ensemble as ens
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor, NearestNeighbors
import sklearn.model_selection as ms
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize


# string workings
import string
from operator import itemgetter
from collections import Counter, OrderedDict

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

# Amazon ML
import boto3
import sagemaker
from sagemaker import get_execution_role

## Load kaggle database

In [60]:
df_reds = pd.read_csv(os.path.join("Data", "winemag-data_first150k.csv"))
print(len(df_reds))
df_reds.drop(['Unnamed: 0'], axis=1, inplace = True)
df_reds['name'] = df_reds['winery'] + ' ' + df_reds['variety']
df_reds.head()

150930


,country,description,designation,points,price,province,region_1,region_2,variety,winery,name
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,Heitz Cabernet Sauvignon
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,Bodega Carmen Rodríguez Tinta de Toro
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,Macauley Sauvignon Blanc
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,Ponzi Pinot Noir
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,Domaine de la Bégude Provence red blend


In [61]:
# sample descrption
df_reds.description[0]

'This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.'

In [62]:
# split data 
desc_train, desc_test, name_train, name_test = train_test_split(df_reds['description'],df_reds['name'], test_size=0.2, random_state=42)
df_des_test = pd.DataFrame(list(zip(name_test, desc_test)),columns =['name', 'description'])

In [63]:
#df_des = df_reds[['name','description']]
df_des = pd.DataFrame(list(zip(name_train, desc_train)),columns =['name', 'description'])
df_des.head()

,name,description
0,Bonterra Sauvignon Blanc,"Nicely dry and crisp in clean acidity, this Sa..."
1,FiàNobile Nero d'Avola,"This Nero d'Avola has aromas of earth, black c..."
2,Sada Vermentino,This elegant Vermentino from Tuscany has a lov...
3,Keenan Merlot,"Not many Merlots deserve time in the cellar, b..."
4,Paumanok Cabernet Sauvignon,"Crisp, elegant black-plum and cassis flavors a..."


## Pre processing text descriptions

In [64]:
all_descriptions = df_des["description"].tolist()
all_descriptions = [des for des in all_descriptions]
#all_descriptions = [item for sublist in flavors_list for item in sublist]
#print(all_descriptions[1])

# Manipulate text
full_corpus = ' '.join(all_descriptions)
sentences_tokenized = sent_tokenize(full_corpus)
stop_words = set(stopwords.words('english')) 
punctuation_table = str.maketrans({key: None for key in string.punctuation})
sno = SnowballStemmer('english')

def normalize_text(raw_text):
    try:
        word_list = word_tokenize(raw_text)
        normalized_sentence = []
        for w in word_list:
            try:
                w = str(w)
                lower_case_word = str.lower(w)
                stemmed_word = sno.stem(lower_case_word)
                no_punctuation = stemmed_word.translate(punctuation_table)
                if len(no_punctuation) > 1 and no_punctuation not in stop_words:
                    normalized_sentence.append(no_punctuation)
            except:
                continue
        return normalized_sentence
    except:
        return ''

In [65]:
print(sentences_tokenized[:10])

['Nicely dry and crisp in clean acidity, this Sauvignon Blanc has flavors of limes, gooseberries, and vanilla.', "But there's a lot of that notorious feline spray aroma and taste that detracts.", "This Nero d'Avola has aromas of earth, black currants, game and a note of grilled bell peppers.", 'The palate offers fleeting black cherry and black berry, accented by black pepper and a green note of dried sage.', 'It finishes on a bitter note.', 'This elegant Vermentino from Tuscany has a lovely floral fragrance of white flowers and stone fruit accompanied by succulent peach and ripe pear flavors.', 'The fruit richness is balanced by crisp freshness.', 'Not many Merlots deserve time in the cellar, but this one does.', "Such are its mountain tannins that it's in lockdown mode, with a tough, astringent finish.", 'But just below the surface are voluptuously ripe flavors of blackberries, cherries and mulberries.']


In [66]:
normalized_sentences = []
for s in sentences_tokenized:
    normalized_text = normalize_text(s)
    normalized_sentences.append(normalized_text)

phrases = Phrases(normalized_sentences)
phrases = Phrases(phrases[normalized_sentences])

ngrams = Phraser(phrases)

phrased_sentences = []
for sent in normalized_sentences:
    phrased_sentence = ngrams[sent]
    phrased_sentences.append(phrased_sentence)

full_list_words = [item for sublist in phrased_sentences for item in sublist]

In [68]:
descriptor_mapping = pd.read_csv(os.path.join("Data", "descriptor_mapping.csv")).set_index('raw descriptor')

def return_mapped_descriptor(word):
    if word in list(descriptor_mapping.index):
        normalized_word = descriptor_mapping['level_3'][word]
        return normalized_word
    else:
        return word

normalized_sentences = []
for sent in phrased_sentences:
    normalized_sentence = []
    for word in sent:
        normalized_word = return_mapped_descriptor(word)
        normalized_sentence.append(str(normalized_word))
    normalized_sentence.append('.')
    normalized_sentence_concat = ' '.join(normalized_sentence)
    normalized_sentences.append(normalized_sentence_concat)

## Train model

In [69]:
# Store training data
with open('wine_corpus.txt', 'w') as f:
    for item in normalized_sentences:
        f.write("{}\n".format(item))


Uploaded this data to AWS sagemaker blazingtext algo and trained the algorithim. Below we load in this training model.

In [71]:
!tar -xvzf dw_model.tar.gz

x vectors.txt
x eval.json
x vectors.bin


In [72]:
num_points = len(open('vectors.txt','r').read().split('\n'))

first_line = True
index_to_word = []
with open("vectors.txt","r") as f:
    for line_num, line in enumerate(f):
        if first_line:
            dim = int(line.strip().split()[1])
            word_vecs = np.zeros((num_points, dim), dtype=float)
            first_line = False
            continue
        line = line.strip()
        word = line.split()[0]
        vec = word_vecs[line_num-1]
        for index, vec_val in enumerate(line.split()[1:]):
            vec[index] = float(vec_val)
        index_to_word.append(word)
        if line_num >= num_points:
            break
word_vecs = normalize(word_vecs, copy=False, return_norm=False)

names_vecs = list(zip(index_to_word, word_vecs))

names_vecs_filtered = [n for n in names_vecs if n[0] in list(descriptor_mapping['level_3'])]

names_vecs_df = pd.DataFrame(names_vecs_filtered, columns=['word', 'vector'])
names_vecs_df.sort_values(by=['word'], inplace=True)
names_vecs_df.to_csv('word_vectors.csv')